In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# URL do dataset HateBR no GitHub
DATASET_URL = 'https://raw.githubusercontent.com/franciellevargas/HateBR/main/dataset/HateBR.csv'

# 1. Carregar o dataset
print("Carregando o dataset...")
try:
    df = pd.read_csv(DATASET_URL)
    print("Dataset carregado com sucesso!")
    # Mostra as primeiras linhas e a distribuição das classes
    display(df.head())
    print("\nDistribuição das classes:")
    print(df['label_final'].value_counts(normalize=True))
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    exit()

# 2. Pré-processamento e Definição das variáveis
print("\nIniciando pré-processamento...")
# Para este MVP, a única limpeza será converter para minúsculas.
# O TfidfVectorizer já lida com muita coisa.
X = df['comentario'].str.lower()
y = df['label_final']

# 3. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

# 4. Vetorização do texto usando TF-IDF
print("Vetorizando o texto...")
vectorizer = TfidfVectorizer(max_features=5000) # Usamos as 5000 palavras mais relevantes

# Aprende o vocabulário com os dados de treino e transforma os dados de treino
X_train_vect = vectorizer.fit_transform(X_train)

# Apenas transforma os dados de teste com o vocabulário já aprendido
X_test_vect = vectorizer.transform(X_test)

# 5. Treinamento do modelo de Regressão Logística
print("Treinando o modelo de classificação...")
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vect, y_train)
print("Modelo treinado com sucesso!")

# 6. Avaliação do modelo
print("\nAvaliando o modelo nos dados de teste...")
y_pred = model.predict(X_test_vect)

print("\nAcurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Não Odioso', 'Odioso']))

# Agora que o modelo está treinado, podemos usá-lo.
# Os objetos que precisamos salvar/usar para novas previsões são:
# - `model` (o classificador)
# - `vectorizer` (o vetorizador)

print("\n--- MVP PRONTO PARA USO ---")

Carregando o dataset...
Dataset carregado com sucesso!


,id,comentario,anotator1,anotator2,anotator3,label_final,links_post,account_post
0,1,Mais um lixo,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
1,2,Essa nao tem vergonha na cara!!,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
2,3,Essa mulher é doente.pilantra!,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
3,4,Comunista safada...,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
4,5,Vagabunda. Comunista. Mentirosa. O povo chilen...,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli



Distribuição das classes:
label_final
1    0.5
0    0.5
Name: proportion, dtype: float64

Iniciando pré-processamento...
Dados divididos: 5600 para treino, 1400 para teste.
Vetorizando o texto...
Treinando o modelo de classificação...
Modelo treinado com sucesso!

Avaliando o modelo nos dados de teste...

Acurácia: 0.8314285714285714

Relatório de Classificação:
              precision    recall  f1-score   support

  Não Odioso       0.83      0.83      0.83       700
      Odioso       0.83      0.83      0.83       700

    accuracy                           0.83      1400
   macro avg       0.83      0.83      0.83      1400
weighted avg       0.83      0.83      0.83      1400


--- MVP PRONTO PARA USO ---


In [3]:
def avaliar_toxicidade(comentario: str, model, vectorizer) -> dict:
    """
    Recebe um comentário e retorna a classificação de toxicidade
    e a probabilidade de ser discurso de ódio.
    """
    # 1. Aplicar o mesmo pré-processamento (minúsculas)
    comentario_processado = comentario.lower()
    
    # 2. Vetorizar o comentário usando o vetorizador JÁ TREINADO
    comentario_vect = vectorizer.transform([comentario_processado])
    
    # 3. Fazer a predição
    predicao = model.predict(comentario_vect)
    probabilidades = model.predict_proba(comentario_vect)
    
    # A probabilidade de ser discurso de ódio é a probabilidade da classe "1"
    prob_odio = probabilidades[0][1]
    
    if predicao[0] == 1:
        classificacao = "Discurso de Ódio"
    else:
        classificacao = "Não é Discurso de Ódio"
        
    return {
        "classificacao": classificacao,
        "nivel_toxicidade": f"{prob_odio:.2%}" # Formata como porcentagem
    }

In [4]:
# --- EXEMPLOS DE USO ---
print("\n--- Testando o MVP com novos comentários ---")

# Exemplo 1: Comentário potencialmente tóxico
comentario1 = "Esses políticos são todos uns bandidos, tinham que sumir do mapa!"
resultado1 = avaliar_toxicidade(comentario1, model, vectorizer)
print(f"Comentário: '{comentario1}'")
print(f"Resultado: {resultado1}\n")

# Exemplo 2: Comentário neutro
comentario2 = "O jogo de futebol ontem foi muito emocionante, gostei bastante do resultado."
resultado2 = avaliar_toxicidade(comentario2, model, vectorizer)
print(f"Comentário: '{comentario2}'")
print(f"Resultado: {resultado2}\n")


--- Testando o MVP com novos comentários ---
Comentário: 'Esses políticos são todos uns bandidos, tinham que sumir do mapa!'
Resultado: {'classificacao': 'Discurso de Ódio', 'nivel_toxicidade': '88.81%'}

Comentário: 'O jogo de futebol ontem foi muito emocionante, gostei bastante do resultado.'
Resultado: {'classificacao': 'Não é Discurso de Ódio', 'nivel_toxicidade': '35.20%'}



In [5]:
# (Após o bloco de treinamento e avaliação do modelo...)
import joblib

print("\nSalvando o modelo e o vetorizador em arquivos...")

# Salva o modelo treinado
joblib.dump(model, 'modelo_odio.joblib')

# Salva o vetorizador
joblib.dump(vectorizer, 'vetorizador_odio.joblib')

print("Modelo e vetorizador salvos com sucesso!")


Salvando o modelo e o vetorizador em arquivos...
Modelo e vetorizador salvos com sucesso!


In [7]:
import praw
import joblib
from dotenv import load_dotenv
import os

# --- CONFIGURAÇÃO ---

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
USER_AGENT = os.getenv("USER_AGENT")

# Carregar o modelo e o vetorizador salvos
try:
    model = joblib.load('modelo_odio.joblib')
    vectorizer = joblib.load('vetorizador_odio.joblib')
    print("Modelo e vetorizador carregados com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivos 'modelo_odio.joblib' ou 'vetorizador_odio.joblib' não encontrados.")
    print("Certifique-se de executar o script de treinamento ('analisador_odio.py') primeiro.")
    exit()

# --- FUNÇÕES ---

def avaliar_toxicidade(comentario: str) -> dict:
    """
    Recebe um comentário e retorna a classificação de toxicidade
    e a probabilidade de ser discurso de ódio.
    """
    comentario_processado = comentario.lower()
    comentario_vect = vectorizer.transform([comentario_processado])
    predicao = model.predict(comentario_vect)
    probabilidades = model.predict_proba(comentario_vect)
    prob_odio = probabilidades[0][1]
    
    return {
        "eh_odio": predicao[0] == 1,
        "nivel_toxicidade": prob_odio
    }

# --- EXECUÇÃO PRINCIPAL ---

def main():
    print("\n--- INICIANDO ANÁLISE DE TOXICIDADE NO REDDIT ---")

    # Conectar à API do Reddit
    try:
        reddit = praw.Reddit(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            user_agent=USER_AGENT,
            check_for_async=False
        )
        print(f"Conexão com a API do Reddit estabelecida. Modo Read-Only: {reddit.read_only}")
    except Exception as e:
        print(f"Erro ao conectar com a API do Reddit: {e}")
        return

    subreddit_alvo = "brasil"
    limite_comentarios = 100

    print(f"\nBuscando posts populares no r/{subreddit_alvo}...")
    
    subreddit = reddit.subreddit(subreddit_alvo)
    try:
        post_popular = next(p for p in subreddit.hot(limit=5) if not p.stickied)
        print(f"Analisando comentários do post: '{post_popular.title}'")
    except StopIteration:
        print(f"Não foi possível encontrar um post válido no r/{subreddit_alvo}.")
        return
    except Exception as e:
        print(f"Ocorreu um erro ao buscar o post: {e}")
        return

    total_toxicidade = 0
    comentarios_odiosos = 0
    comentarios_analisados = 0
    
    # NOVO: Lista para armazenar os comentários classificados como ódio
    comentarios_odiosos_lista = []

    post_popular.comments.replace_more(limit=0)

    for comment in post_popular.comments.list():
        if comentarios_analisados >= limite_comentarios:
            break
        
        texto_comentario = comment.body
        
        if not texto_comentario or texto_comentario in ['[deleted]', '[removed]']:
            continue

        resultado = avaliar_toxicidade(texto_comentario)
        total_toxicidade += resultado["nivel_toxicidade"]
        
        if resultado["eh_odio"]:
            comentarios_odiosos += 1
            # NOVO: Adiciona o texto do comentário à lista
            comentarios_odiosos_lista.append(texto_comentario)
        
        comentarios_analisados += 1

    media_toxicidade = (total_toxicidade / comentarios_analisados) if comentarios_analisados > 0 else 0
    
    print("\n--- Relatório Final da Análise ---")
    print(f"  - Subreddit analisado: r/{subreddit_alvo}")
    print(f"  - Post: '{post_popular.title[:60]}...'")
    print(f"  - Comentários analisados: {comentarios_analisados}")
    print(f"  - Comentários classificados como discurso de ódio: {comentarios_odiosos}")
    print(f"  - Nível médio de toxicidade nos comentários: {media_toxicidade:.2%}")

    # NOVO: Loop para imprimir os comentários ofensivos encontrados
    print("\n" + "="*50) # Adiciona uma linha separadora

    if comentarios_odiosos_lista:
        print("\n--- Comentários Classificados como Discurso de Ódio ---")
        for i, comentario in enumerate(comentarios_odiosos_lista, 1):
            print(f"{i}. {comentario}\n---") # Adiciona um separador entre os comentários
    else:
        print("\nNenhum comentário foi classificado como discurso de ódio nesta amostra.")


if __name__ == "__main__":
    main()

Modelo e vetorizador carregados com sucesso.

--- INICIANDO ANÁLISE DE TOXICIDADE NO REDDIT ---
Conexão com a API do Reddit estabelecida. Modo Read-Only: True

Buscando posts populares no r/brasil...
Analisando comentários do post: 'Arte de Laerte'

--- Relatório Final da Análise ---
  - Subreddit analisado: r/brasil
  - Post: 'Arte de Laerte...'
  - Comentários analisados: 30
  - Comentários classificados como discurso de ódio: 14
  - Nível médio de toxicidade nos comentários: 49.63%


--- Comentários Classificados como Discurso de Ódio ---
1. o traço dela é tão foda que ela desenha a coisa mais manjada e óbvia do mundo e eu ainda pago pau
---
2. #"O crime está em todo lugar.... Até dentro dessa casa."
---
3. Detalhe... Essa charge é de Outubro de 2024... O Chupetinha veio a público defender Fintech de PCC poucos meses depois. Todo mundo já sabia o que tava rolando.
---
4. Na época a maior apreensão de fuzis no RJ foi no condomínio do Bolsonaro.

PM jagunço do Caiado matou informante 

In [2]:
import praw
import joblib
from datetime import datetime, timezone
import pandas as pd
import time
from collections import Counter

#TESTAR COM PERFIS = FAUSEN, No-Job-193, Low-Stay-5562, Madrugada123,


def analisar_perfil_usuario(username, reddit, model, vectorizer, limite=100):
    """
    Analisa o perfil de um usuário do Reddit baseado em seus últimos comentários
    
    Args:
        username (str): Nome do usuário do Reddit (sem u/) - Unica coisa que precisa preencher de vdd
        reddit: Instância do PRAW
        model: Modelo treinado de classificação
        vectorizer: Vetorizador TF-IDF treinado
        limite (int): Número máximo de comentários para analisar
    
    Returns:
        dict: Análise completa do perfil do usuário
    """
    
    def avaliar_toxicidade_local(comentario):
        comentario_processado = comentario.lower()
        comentario_vect = vectorizer.transform([comentario_processado])
        predicao = model.predict(comentario_vect)
        probabilidades = model.predict_proba(comentario_vect)
        prob_odio = probabilidades[0][1]
        
        return {
            "odio": predicao[0] == 1,
            "nivel_toxicidade": prob_odio
        }
    
    try:
        print(f"Buscando dados do usuário: {username}")
        user = reddit.redditor(username)
        
        # Verifica se o usuário existe
        try:
            user_created = user.created_utc
        except Exception as e:
            return {
                'erro': f'Usuário {username} não encontrado ou perfil suspenso',
                'detalhes': str(e)
            }
        
        comentarios_analisados = 0
        total_toxicidade = 0
        comentarios_odiosos = 0
        historico_comentarios = []
        subreddits_atividade = []
        comentarios_por_dia = {}
        
        print(f"Analisando até {limite} comentários mais recentes...")
        
        # Busca os comentários do usuário
        try:
            for comment in user.comments.new(limit=limite):
                if comentarios_analisados >= limite:
                    break
                
                # Pula comentários deletados/removidos
                if not comment.body or comment.body in ['[deleted]', '[removed]']:
                    continue
                
                # Analisa a toxicidade do comentário
                resultado = avaliar_toxicidade_local(comment.body)
                total_toxicidade += resultado["nivel_toxicidade"]
                
                if resultado["odio"]:
                    comentarios_odiosos += 1
                
                # Converte timestamp para data legível
                data_comentario = datetime.fromtimestamp(comment.created_utc, tz=timezone.utc)
                data_str = data_comentario.strftime('%Y-%m-%d')
                
                # Registra atividade por dia
                if data_str in comentarios_por_dia:
                    comentarios_por_dia[data_str] += 1
                else:
                    comentarios_por_dia[data_str] = 1
                
                # Registra subreddit de atividade
                subreddits_atividade.append(comment.subreddit.display_name)
                
                # Salva detalhes do comentário
                historico_comentarios.append({
                    'texto': comment.body[:200] + "..." if len(comment.body) > 200 else comment.body,
                    'texto_completo': comment.body,
                    'toxicidade': resultado["nivel_toxicidade"],
                    'odio': resultado["odio"],
                    'subreddit': comment.subreddit.display_name,
                    'data': data_str,
                    'timestamp': comment.created_utc,
                    'score': comment.score,
                    'url': f"https://reddit.com{comment.permalink}"
                })
                
                comentarios_analisados += 1
                
                # Pequena pausa para não sobrecarregar a API
                if comentarios_analisados % 10 == 0:
                    print(f"Analisados {comentarios_analisados} comentários...")
                    time.sleep(0.1)
                    
        except Exception as e:
            print(f"Erro ao buscar comentários: {e}")
            if comentarios_analisados == 0:
                return {
                    'erro': f'Não foi possível acessar os comentários de {username}',
                    'detalhes': 'Perfil pode ser privado ou sem comentários públicos'
                }
        
        if comentarios_analisados == 0:
            return {
                'erro': f'Usuário {username} não possui comentários públicos para analisar'
            }
        
        # Calcula estatísticas
        nivel_medio_toxicidade = total_toxicidade / comentarios_analisados
        percentual_odioso = (comentarios_odiosos / comentarios_analisados) * 100
        
        # Analisa subreddits mais ativos
        subreddits_counter = Counter(subreddits_atividade)
        top_subreddits = subreddits_counter.most_common(10)
        
        # Encontra os comentários mais tóxicos
        comentarios_ordenados = sorted(historico_comentarios, 
                                     key=lambda x: x['toxicidade'], 
                                     reverse=True)
        comentarios_mais_toxicos = comentarios_ordenados[:5]
        
        # Calcula karma médio dos comentários
        scores = [c['score'] for c in historico_comentarios if c['score'] is not None]
        karma_medio = sum(scores) / len(scores) if scores else 0
        
        # Data de criação da conta
        data_criacao = datetime.fromtimestamp(user_created, tz=timezone.utc)
        dias_desde_criacao = (datetime.now(timezone.utc) - data_criacao).days
        
        # Classifica o usuário
        if percentual_odioso >= 50:
            classificacao = "MUITO TÓXICO"
            cor_classificacao = "🔴"
        elif percentual_odioso >= 30:
            classificacao = "MODERADAMENTE TÓXICO"
            cor_classificacao = "🟡"
        elif percentual_odioso >= 10:
            classificacao = "LEVEMENTE TÓXICO"
            cor_classificacao = "🟠"
        else:
            classificacao = "NÃO TÓXICO"
            cor_classificacao = "🟢"
        
        resultado_final = {
            'username': username,
            'classificacao': classificacao,
            'cor_classificacao': cor_classificacao,
            'resumo': {
                'total_comentarios_analisados': comentarios_analisados,
                'comentarios_odiosos': comentarios_odiosos,
                'percentual_odioso': round(percentual_odioso, 2),
                'nivel_medio_toxicidade': round(nivel_medio_toxicidade, 4),
                'karma_medio_comentarios': round(karma_medio, 2)
            },
            'perfil': {
                'data_criacao_conta': data_criacao.strftime('%Y-%m-%d'),
                'dias_desde_criacao': dias_desde_criacao,
                'karma_comentarios': user.comment_karma,
                'karma_posts': user.link_karma
            },
            'atividade': {
                'subreddits_mais_ativos': top_subreddits,
                'comentarios_por_dia': comentarios_por_dia,
                'periodo_analisado': f"{min(comentarios_por_dia.keys())} até {max(comentarios_por_dia.keys())}" if comentarios_por_dia else "N/A"
            },
            'comentarios_mais_toxicos': comentarios_mais_toxicos,
            'historico_completo': historico_comentarios
        }
        
        return resultado_final
        
    except Exception as e:
        return {
            'erro': f'Erro inesperado ao analisar {username}',
            'detalhes': str(e)
        }

def imprimir_relatorio_usuario(resultado):
    
    if 'erro' in resultado:
        print(f"!!!! ERRO !!!!: {resultado['erro']}")
        if 'detalhes' in resultado:
            print(f"Detalhes: {resultado['detalhes']}")
        return
    
    print("\n" + "="*80)
    print(f" RELATÓRIO DE ANÁLISE DE TOXICIDADE - USUÁRIO: u/{resultado['username']}")
    print("="*80)

    
    print(f"\n{resultado['cor_classificacao']} CLASSIFICAÇÃO: {resultado['classificacao']}")
    print("-"*80)
    
    print(f"\n RESUMO ESTATÍSTICO:")
    print(f"  • Comentários analisados: {resultado['resumo']['total_comentarios_analisados']}")
    print(f"  • Comentários ofensivos: {resultado['resumo']['comentarios_odiosos']}")
    print(f"  • Percentual de toxicidade: {resultado['resumo']['percentual_odioso']:.2f}%")
    print(f"  • Nível médio de toxicidade: {resultado['resumo']['nivel_medio_toxicidade']:.2f}")
    print(f"  • Karma médio por comentário: {resultado['resumo']['karma_medio_comentarios']:.1f}")
    
    print("-"*80)
    print(f"\n INFORMAÇÕES DO PERFIL:")
    print(f"  • Conta criada em: {resultado['perfil']['data_criacao_conta']}")
    print(f"  • Dias desde criação: {resultado['perfil']['dias_desde_criacao']}")
    print(f"  • Karma de comentários: {resultado['perfil']['karma_comentarios']:,}")
    print(f"  • Karma de posts: {resultado['perfil']['karma_posts']:,}")
    print("-"*80)
    
    print(f"\n ATIVIDADE POR SUBREDDIT (Top 5):")
    for i, (subreddit, count) in enumerate(resultado['atividade']['subreddits_mais_ativos'][:5], 1):
        print(f"  {i}. r/{subreddit}: {count} comentários")

    print("-"*80)
    
    print(f"\n COMENTÁRIOS MAIS TÓXICOS:")
    for i, comentario in enumerate(resultado['comentarios_mais_toxicos'][:3], 1):
        print(f"\n  {i}. Toxicidade: {comentario['toxicidade']:.2f} | r/{comentario['subreddit']}")
        print(f"     \"{comentario['texto']}\"")
        print(f"     URL: {comentario['url']}")
    
    print(f"\n PERÍODO ANALISADO: {resultado['atividade']['periodo_analisado']}")
    print("="*80)

# Exemplo de uso
def exemplo_uso():
    """
    Exemplo de como usar as funções
    """
    import os
    from dotenv import load_dotenv
    
    load_dotenv()
    
    # Configuração do Reddit
    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    USER_AGENT = os.getenv("USER_AGENT")
    
    # Carrega o modelo
    try:
        model = joblib.load('modelo_odio.joblib')
        vectorizer = joblib.load('vetorizador_odio.joblib')
    except FileNotFoundError:
        print("ERRO: Modelos não encontrados. Execute o treinamento primeiro.")
        return
    
    # Conecta ao Reddit
    reddit = praw.Reddit(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        user_agent=USER_AGENT,
        check_for_async=False
    )
    
    # Analisa um usuário (substitua pelo username desejado)
    username = input("Digite o username para analisar (sem u/): ")
    
    resultado = analisar_perfil_usuario(username, reddit, model, vectorizer, limite=100)
    imprimir_relatorio_usuario(resultado)
    
    return resultado

if __name__ == "__main__":
    exemplo_uso()

Buscando dados do usuário:  Low-Stay-5562
Analisando até 100 comentários mais recentes...
Analisados 10 comentários...
Analisados 20 comentários...
Analisados 30 comentários...
Analisados 40 comentários...
Analisados 50 comentários...
Analisados 60 comentários...
Analisados 70 comentários...
Analisados 80 comentários...
Analisados 90 comentários...
Analisados 100 comentários...

 RELATÓRIO DE ANÁLISE DE TOXICIDADE - USUÁRIO: u/ Low-Stay-5562

🟡 CLASSIFICAÇÃO: MODERADAMENTE TÓXICO
--------------------------------------------------------------------------------

 RESUMO ESTATÍSTICO:
  • Comentários analisados: 100
  • Comentários ofensivos: 41
  • Percentual de toxicidade: 41.00%
  • Nível médio de toxicidade: 0.50
  • Karma médio por comentário: 12.8
--------------------------------------------------------------------------------

 INFORMAÇÕES DO PERFIL:
  • Conta criada em: 2022-01-18
  • Dias desde criação: 1320
  • Karma de comentários: 4,998
  • Karma de posts: 196
-----------------